# Imports,  Settings and Paths

In [1]:
comment = 'bn_shuffle_adam_abc'

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd#
%matplotlib inline
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time

In [3]:
from Code.Models.c7o2h10_model import DeepPotential, train, normalize, backtransform
from Code.DataGeneration.saver import create_path
from Code.Models.nn_extentions import AbcExponentialLR

In [4]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
model_path = './ModelCheckpoints/c7o2h10/{}/'.format(comment)
create_path(model_path)

In [5]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [6]:
use_cuda = torch.cuda.is_available()
print('use cuda:  ', use_cuda)

use cuda:   True


# Read the Data

In [7]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1
Y_npy, Y_min, Y_max = normalize(Y_npy)
X_npy.shape

(404000, 19, 72)

### Check for NaN values

In [8]:
use_ids = np.where(np.isnan(X_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [9]:
X_npy = X_npy[use_ids]
Y_npy = Y_npy[use_ids]
X_npy.shape

(404000, 19, 72)

In [10]:
X_npy = X_npy
Y_npy = Y_npy

### Split into test and train

In [11]:
X_train_npy, X_test_npy = X_npy[train_ids], X_npy[test_ids]
Y_train_npy, Y_test_npy = Y_npy[train_ids], Y_npy[test_ids]
print('X Train shape: {}\tX Test shape: {}\nY Train shape: {}\tY Test shape: {}'.format(X_train_npy.shape,
                                                                                        X_test_npy.shape,
                                                                                        Y_train_npy.shape,
                                                                                        Y_test_npy.shape))

X Train shape: (400000, 19, 72)	X Test shape: (4000, 19, 72)
Y Train shape: (400000,)	Y Test shape: (4000,)


### Shuffle the Dataset

In [12]:
#shuffle = np.arange(X_train_npy.shape[0])
#np.random.shuffle(shuffle)
#X_train_npy = X_train_npy[shuffle]
#Y_train_npy = Y_train_npy[shuffle]

### Create Pytorch Variables

In [13]:
if use_cuda:
    X_train = Variable(torch.Tensor(X_train_npy).cuda())
    Y_train = Variable(torch.Tensor(Y_train_npy).cuda(), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy).cuda())
    Y_test = Variable(torch.Tensor(Y_test_npy).cuda(), requires_grad=False)
else:
    X_train = Variable(torch.Tensor(X_train_npy))
    Y_train = Variable(torch.Tensor(Y_train_npy), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy))
    Y_test = Variable(torch.Tensor(Y_test_npy), requires_grad=False)

/home/mi/stefaah94/miniconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU1 NVS 310 which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


# Train Model

## Do the learning

In [ ]:
if use_cuda:
    init_deep_pot = DeepPotential().cuda()
else:
    init_deep_pot = DeepPotential()
init_optim = torch.optim.Adam(init_deep_pot.parameters(), lr=0.01)
lr_scheduler = AbcExponentialLR(init_optim, 0.96, 1.5)
deep_pot, optim = train(init_deep_pot, init_optim, lr_scheduler, X_train, Y_train,
                        X_test, Y_test,
                        300, 128,
                        checkpoint_path=model_path, print_every=100, shuffle=False)

total: 0.0 %	current epoch: 3.2 %	loss: 0.011185	time estimate: 621.6 min
total: 0.0 %	current epoch: 6.4 %	loss: 0.014692	time estimate: 537.4 min
total: 0.0 %	current epoch: 9.6 %	loss: 0.006766	time estimate: 593.7 min
total: 0.0 %	current epoch: 12.8 %	loss: 0.008277	time estimate: 613.3 min
total: 0.1 %	current epoch: 16.0 %	loss: 0.007032	time estimate: 628.0 min
total: 0.1 %	current epoch: 19.2 %	loss: 0.00571	time estimate: 612.5 min
total: 0.1 %	current epoch: 22.4 %	loss: 0.007796	time estimate: 601.3 min
total: 0.1 %	current epoch: 25.6 %	loss: 0.007497	time estimate: 591.3 min
total: 0.1 %	current epoch: 28.8 %	loss: 0.008774	time estimate: 590.3 min
total: 0.1 %	current epoch: 32.0 %	loss: 0.004917	time estimate: 602.6 min
total: 0.1 %	current epoch: 35.2 %	loss: 0.007235	time estimate: 609.8 min
total: 0.1 %	current epoch: 38.4 %	loss: 0.004587	time estimate: 601.8 min
total: 0.1 %	current epoch: 41.6 %	loss: 0.00314	time estimate: 595.0 min
total: 0.1 %	current epoch: 44

In [ ]:
#deep_pot = DeepPotential().cuda()
#deep_pot.load_state_dict(torch.load('./ModelCheckpoints/epoch_13'))
Y_result = deep_pot.forward(X_test)

In [ ]:
result = backtransform(np.array(Y_result.data.tolist()), Y_min, Y_max)

In [ ]:
test_labels = backtransform(np.array(Y_test), Y_min, Y_max)

In [ ]:
np.mean(np.abs(test_labels - result))

In [ ]:
test_labels[:10]

In [ ]:
result[:10]

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
test_result = np.squeeze(backtransform(test_model(X_test), Y_min, Y_max).detach().cpu().numpy())

In [ ]:
val_result = np.squeeze(backtransform(Y_test, Y_min, Y_max).detach().cpu().numpy())

In [ ]:
np.mean(np.abs(test_result - val_result))

In [ ]:
val_result[:10]

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)